In [1]:
import torch.nn as nn
import torch.nn.functional as F
import math

class Linear(nn.Module):

    def __init__(self, num_classes=2):

        super(Linear, self).__init__()

        self.number_class   = num_classes

        _size_image     = 100* 100
        _num1           = 50
        _num2           = 60
        
        self.fc1        = nn.Linear(_size_image, _num1, bias=True)
        self.fc2        = nn.Linear(_num1, _num2, bias=True)
        self.fc3        = nn.Linear(_num2, num_classes, bias=True)

        self.fc_layer1  = nn.Sequential(self.fc1, nn.LeakyReLU(True))
        self.fc_layer2  = nn.Sequential(self.fc2, nn.LeakyReLU(True))
        self.fc_layer3  = nn.Sequential(self.fc3, nn.Sigmoid())
        
        self.classifier = nn.Sequential(self.fc_layer1, self.fc_layer2, self.fc_layer3)
        
        self._initialize_weight()        
        
    def _initialize_weight(self):

        for name, m in self._modules.items():
            if isinstance(m, nn.Linear):
                n = m.in_features
                m.weight.data.uniform_(- 1.0 / math.sqrt(n), 1.0 / math.sqrt(n))

                if m.bias is not None:
                    m.bias.data.zero_()

    def forward(self, x):

        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x


In [2]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import argparse
import sys
import os
import numpy as np
import time
import datetime 
import csv
import configparser
import argparse
import platform
import torchvision.transforms as transforms
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
from random import shuffle

# -----------------------------------------------------------------------------
# load dataset
# -----------------------------------------------------------------------------


transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])


#train_data_path = 'relative path of training data set'
train_data_path = './horse-or-human/horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=3, shuffle=True, num_workers=1)  


validation_data_path = './horse-or-human/horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=3, shuffle=True, num_workers=1)


In [3]:
num_classes = 2

# -----------------------------------------------------------------------------
# load neural network model
# -----------------------------------------------------------------------------

model = Linear(num_classes=num_classes)

# -----------------------------------------------------------------------------
# Set the flag for using cuda
# -----------------------------------------------------------------------------

bCuda = 0

if bCuda:
 
    model.cuda()

# -----------------------------------------------------------------------------
# optimization algorithm
# -----------------------------------------------------------------------------

optimizer   = optim.SGD(model.parameters(), lr=0.015, weight_decay=0.02)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=1, min_lr=2e-4, verbose=1)
objective   = nn.CrossEntropyLoss()

# -----------------------------------------------------------------------------
# function for training the model
# -----------------------------------------------------------------------------

def train():

    # print('train the model at given epoch')
    accuracy_train = []
    loss_train = []
    correct = 0
    correct_batch = 0

    model.train()
    for idx_batch, (data, target) in enumerate(trainloader):

        if bCuda:
        
            data, target    = data.cuda(), target.cuda()

        data, target    = Variable(data), Variable(target)

        optimizer.zero_grad()

        output  = model(data)
        loss    = objective(output, target)

        loss.backward()
        optimizer.step()

        loss_train_batch    = loss.item() / len(data)
        loss_train.append(loss_train_batch)
        pred        = output.data.max(1)[1]
        correct     += pred.eq(target.data.view_as(pred)).cpu().sum()
        correct_batch = pred.eq(target.data.view_as(pred)).cpu().sum()
        accuracy_train_batch = float(correct_batch) / len(data)
        accuracy_train.append(accuracy_train_batch)
        
    loss_train_mean     = np.mean(loss_train)
    loss_train_std      = np.std(loss_train)
    accuracy_train_mean   = float(correct) / len(trainloader.dataset)  # mean accuracy

    return {'loss_train_mean': loss_train_mean, 'loss_train_std': loss_train_std, 'accuracy_train_mean' : accuracy_train_mean}

# -----------------------------------------------------------------------------
# function for testing the model
# -----------------------------------------------------------------------------

def test():

    # print('test the model at given epoch')

    accuracy_test   = []
    loss_test       = 0
    correct         = 0

    model.eval()

    for idx_batch, (data, target) in enumerate(valloader):

        if bCuda:
        
            data, target    = data.cuda(), target.cuda()

        data, target    = Variable(data), Variable(target)

        output  = model(data)
        loss    = objective(output, target)

        loss_test   += loss.item()
        pred        = output.data.max(1)[1]
        correct     += pred.eq(target.data.view_as(pred)).cpu().sum()

    loss_test       = loss_test / len(valloader.dataset)
    accuracy_test   = 100. * float(correct) / len(valloader.dataset)

    return {'loss_test': loss_test, 'accuracy_test': accuracy_test}

# -----------------------------------------------------------------------------
# iteration for the epoch
# -----------------------------------------------------------------------------

loss_train_mean, loss_train_std, loss_test, accuracy_test = [], [], [], []
loss_temp = 0;

for e in range(3000):
    
    result_train    = train()
    result_test     = test()
    scheduler.step(result_test['loss_test'], e)

    
    loss_train_mean.append(result_train['loss_train_mean'])
    loss_train_std.append(result_train['loss_train_std'])
    loss_test.append(result_test['loss_test'])
    accuracy_test.append(result_test['accuracy_test'])
    
    print(f"[epoch {e}] loss: (training) {loss_train_mean[-1]} , (testing) {loss_test[-1]} , accuracy: (training) {result_train['accuracy_train_mean']} ,    (testing) {accuracy_test[-1]}")
    
    if(abs(loss_temp - loss_train_mean[-1]) < 10e-7):
        break;
    loss_temp = loss_train_mean[-1]


[epoch 0] loss: (training) 0.21917209799597867 , (testing) 0.18558936880435795 , accuracy: (training) 0.6105160662122687 ,    (testing) 74.609375
[epoch 1] loss: (training) 0.19766214009160782 , (testing) 0.15269093750976026 , accuracy: (training) 0.7147030185004869 ,    (testing) 87.109375
[epoch 2] loss: (training) 0.18334113506001098 , (testing) 0.15237864118535072 , accuracy: (training) 0.7546251217137293 ,    (testing) 86.71875
[epoch 3] loss: (training) 0.17881103646303406 , (testing) 0.21603672951459885 , accuracy: (training) 0.7779941577409932 ,    (testing) 61.328125
Epoch     4: reducing learning rate of group 0 to 1.5000e-03.
[epoch 4] loss: (training) 0.17273381755002157 , (testing) 0.16047668259125203 , accuracy: (training) 0.7964946445959105 ,    (testing) 82.03125
[epoch 5] loss: (training) 0.16253255882578294 , (testing) 0.15092049597296864 , accuracy: (training) 0.8393378773125608 ,    (testing) 85.15625
[epoch 6] loss: (training) 0.15946532616462375 , (testing) 0.1627

[epoch 56] loss: (training) 0.15287885906860826 , (testing) 0.151406251010485 , accuracy: (training) 0.8851022395326192 ,    (testing) 84.765625
[epoch 57] loss: (training) 0.1528524754693835 , (testing) 0.15139210980851203 , accuracy: (training) 0.8841285296981499 ,    (testing) 84.765625
[epoch 58] loss: (training) 0.15276110068007043 , (testing) 0.15629911608994007 , accuracy: (training) 0.8831548198636806 ,    (testing) 82.03125
[epoch 59] loss: (training) 0.15275961285198164 , (testing) 0.15585563087370247 , accuracy: (training) 0.8821811100292113 ,    (testing) 83.203125
[epoch 60] loss: (training) 0.1526060685935359 , (testing) 0.15354278474114835 , accuracy: (training) 0.8860759493670886 ,    (testing) 83.984375
[epoch 61] loss: (training) 0.1523941448003604 , (testing) 0.14839531620964408 , accuracy: (training) 0.8841285296981499 ,    (testing) 87.109375
[epoch 62] loss: (training) 0.152549750835022 , (testing) 0.15238131512887776 , accuracy: (training) 0.8831548198636806 ,   

[epoch 113] loss: (training) 0.148459455622876 , (testing) 0.15146819490473717 , accuracy: (training) 0.8928919182083739 ,    (testing) 85.15625
[epoch 114] loss: (training) 0.14873855776063902 , (testing) 0.15126525214873254 , accuracy: (training) 0.8958130477117819 ,    (testing) 85.15625
[epoch 115] loss: (training) 0.1487805593118028 , (testing) 0.15323730546515435 , accuracy: (training) 0.8977604673807206 ,    (testing) 82.421875
[epoch 116] loss: (training) 0.14820488053577635 , (testing) 0.15361764922272414 , accuracy: (training) 0.8958130477117819 ,    (testing) 82.421875
[epoch 117] loss: (training) 0.14788141212379968 , (testing) 0.15594801132101566 , accuracy: (training) 0.8977604673807206 ,    (testing) 82.03125
[epoch 118] loss: (training) 0.1482943096012601 , (testing) 0.1497703093336895 , accuracy: (training) 0.9016553067185978 ,    (testing) 86.71875
[epoch 119] loss: (training) 0.1479451055371495 , (testing) 0.15270048682577908 , accuracy: (training) 0.8948393378773125

[epoch 169] loss: (training) 0.144454124283165 , (testing) 0.15786303742788732 , accuracy: (training) 0.9133398247322297 ,    (testing) 80.46875
[epoch 170] loss: (training) 0.1444171902273788 , (testing) 0.15334802167490125 , accuracy: (training) 0.9123661148977604 ,    (testing) 84.375
[epoch 171] loss: (training) 0.14474016653900593 , (testing) 0.15344935143366456 , accuracy: (training) 0.9104186952288218 ,    (testing) 84.375
[epoch 172] loss: (training) 0.1441805732841047 , (testing) 0.15576600166969 , accuracy: (training) 0.9104186952288218 ,    (testing) 82.8125
[epoch 173] loss: (training) 0.14414947619706941 , (testing) 0.1524863630766049 , accuracy: (training) 0.9152872444011685 ,    (testing) 85.15625
[epoch 174] loss: (training) 0.14425070840137694 , (testing) 0.15519708779174834 , accuracy: (training) 0.9133398247322297 ,    (testing) 82.8125
[epoch 175] loss: (training) 0.14413640265098696 , (testing) 0.15280309552326798 , accuracy: (training) 0.9133398247322297 ,    (tes

[epoch 226] loss: (training) 0.14107920427827492 , (testing) 0.15669125400017947 , accuracy: (training) 0.9230769230769231 ,    (testing) 82.8125
[epoch 227] loss: (training) 0.14088779615475439 , (testing) 0.15799647220410407 , accuracy: (training) 0.9211295034079844 ,    (testing) 82.8125
[epoch 228] loss: (training) 0.14131904491182667 , (testing) 0.15450653480365872 , accuracy: (training) 0.9240506329113924 ,    (testing) 83.984375
[epoch 229] loss: (training) 0.1417601254811208 , (testing) 0.1540542122675106 , accuracy: (training) 0.9259980525803311 ,    (testing) 84.375
[epoch 230] loss: (training) 0.14073177364060205 , (testing) 0.15707676357124 , accuracy: (training) 0.9240506329113924 ,    (testing) 82.8125
[epoch 231] loss: (training) 0.14060844091446914 , (testing) 0.15317558485548943 , accuracy: (training) 0.9259980525803311 ,    (testing) 84.375
[epoch 232] loss: (training) 0.14064911184551881 , (testing) 0.1576570450561121 , accuracy: (training) 0.9230769230769231 ,    (t

[epoch 283] loss: (training) 0.13791591525657068 , (testing) 0.1596701688831672 , accuracy: (training) 0.9357351509250244 ,    (testing) 80.859375
[epoch 284] loss: (training) 0.1378990654753063 , (testing) 0.15602177660912275 , accuracy: (training) 0.9406037000973709 ,    (testing) 82.8125
[epoch 285] loss: (training) 0.13782505669329673 , (testing) 0.15542076516430825 , accuracy: (training) 0.9367088607594937 ,    (testing) 83.59375
[epoch 286] loss: (training) 0.13764294034885596 , (testing) 0.14987868361640722 , accuracy: (training) 0.9425511197663097 ,    (testing) 85.9375
[epoch 287] loss: (training) 0.13771741722137518 , (testing) 0.1602373415371403 , accuracy: (training) 0.937682570593963 ,    (testing) 80.46875
[epoch 288] loss: (training) 0.13878288737654804 , (testing) 0.14896144182421267 , accuracy: (training) 0.937682570593963 ,    (testing) 86.328125
[epoch 289] loss: (training) 0.1376785606985769 , (testing) 0.1541588215623051 , accuracy: (training) 0.9406037000973709 , 